In [1]:
#I used the following general methodology for analysis / prediction:

    #Imported CSV and converted to data frame
    #Performed exploratory data analysis
    #Cleansed data
    #Random Forest Classifier algorithm
 
# The model selects a sample of claimants from the original dataset and predicts opiod use each time the script is run
# The script generates the output in a .csv file "opiodusepredictionresults.csv"
#I have used a random forrest classifier model to predict opiod use by claimants







In [2]:
#import libraries

#Multidimensional Array and Matrix Representation Library needed to input cleansed data into model
import numpy as np 

# Python Data Analysis Library for Data Frame, CSV File I/O 
import pandas as pd 

#Data Visualization Library
%matplotlib inline
import matplotlib.pyplot as plt

#Data Visualization Library built on top of Matplotlib
#a cleaner visualization and easier interface to call
import seaborn as sns


#Algorithms and accuracy testing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import export_graphviz


In [3]:
#DATA IMPORT, INSPECTION AND CLEANSING


#i'm going to standardize any missing values in 
#our data frame so that all missing values are picked up i.e. n/a, na, Naan 
missing_values = ["n/a", "na", "--"]

#import file and define as maindf, i've renamed the original file to 'train.csv' 
#for ease of use. This will be the file that will train the model
maindf = pd.read_csv('train.csv',low_memory=False, na_values = missing_values)

In [4]:
#inspect the data by looking at the top 5 rows
maindf.head()

,ClaimID,Accident DateID,Claim Setup DateID,Report To GB DateID,Employer Notification DateID,Benefits State,Accident State,Industry ID,Claimant Age,Claimant Sex,...,CPT Category - Eval_Mgmt,CPT Category - Medicine,CPT Category - Path_Lab,CPT Category - Radiology,CPT Category - Surgery,NDC Class - Benzo,NDC Class - Misc (Zolpidem),NDC Class - Muscle Relaxants,NDC Class - Stimulants,Opiods Used
0,7275,33460.0,33467,33462,33460.0,OK,OK,17,41,M,...,7,1,3,5,1,0,0,0,0,False
1,17596,NaN,34301,34300,34270.0,CO,NaN,17,49,M,...,4,12,1,4,9,0,0,0,0,True
2,13834,35043.0,35054,35050,35043.0,MD,MD,17,47,M,...,4,1,0,0,2,0,0,0,0,False
3,5719,33136.0,33147,33145,33136.0,IL,IL,9,35,F,...,2,1,0,2,0,0,0,1,0,False
4,8646,33262.0,33277,33264,33264.0,IL,IL,9,30,F,...,2,4,0,6,0,0,0,0,0,False


In [5]:
#inspect the data by looking at the bottome 5 rows
maindf.tail()

,ClaimID,Accident DateID,Claim Setup DateID,Report To GB DateID,Employer Notification DateID,Benefits State,Accident State,Industry ID,Claimant Age,Claimant Sex,...,CPT Category - Eval_Mgmt,CPT Category - Medicine,CPT Category - Path_Lab,CPT Category - Radiology,CPT Category - Surgery,NDC Class - Benzo,NDC Class - Misc (Zolpidem),NDC Class - Muscle Relaxants,NDC Class - Stimulants,Opiods Used
160611,182862,33473.0,33475,33473,33473.0,PA,PA,12,50,M,...,3,9,1,4,1,0,0,0,0,False
160612,198379,34793.0,34801,34795,34793.0,PA,PA,7,52,F,...,4,7,0,4,1,0,0,0,0,False
160613,182403,NaN,34902,34901,34901.0,PA,NaN,16,22,M,...,10,21,6,5,4,0,0,0,0,True
160614,190166,33757.0,33761,33760,33759.0,AZ,AZ,7,60,F,...,4,6,0,0,0,0,0,1,0,False
160615,182604,33915.0,33915,33915,33915.0,NJ,NJ,7,50,F,...,3,3,0,0,1,0,0,0,0,False


In [6]:
# statistical description of numerical columns
maindf.describe()

,ClaimID,Accident DateID,Claim Setup DateID,Report To GB DateID,Employer Notification DateID,Industry ID,Claimant Age,Number Dependents,Weekly Wage,Max Medical Improvement DateID,...,CPT Category - Anesthesia,CPT Category - Eval_Mgmt,CPT Category - Medicine,CPT Category - Path_Lab,CPT Category - Radiology,CPT Category - Surgery,NDC Class - Benzo,NDC Class - Misc (Zolpidem),NDC Class - Muscle Relaxants,NDC Class - Stimulants
count,160616.000000,144713.000000,160616.000000,160616.000000,160615.000000,160616.000000,160616.000000,160616.000000,160616.000000,72580.000000,...,160616.000000,160616.000000,160616.000000,160616.000000,160616.000000,160616.000000,160616.000000,160616.000000,160616.000000,160616.000000
mean,499642.652426,34158.445350,34220.719897,34215.138230,34207.019170,11.618905,42.814533,0.427573,709.457650,34396.177804,...,0.318026,3.752615,5.819557,1.765465,2.408558,1.684135,0.007023,0.009999,0.194426,0.000286
std,288365.183875,649.503895,660.237952,660.180925,661.044586,4.321883,13.176824,1.027837,487.976345,638.807697,...,0.698940,2.461367,6.055342,5.241317,2.730262,2.736504,0.111368,0.119508,0.597437,0.022316
min,7.000000,33064.000000,33067.000000,33066.000000,28932.000000,0.000000,0.000000,0.000000,0.000000,33066.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,249379.750000,33602.000000,33656.000000,33650.000000,33642.000000,8.000000,32.000000,0.000000,385.065000,33873.000000,...,0.000000,2.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,499038.500000,34137.000000,34214.000000,34208.000000,34198.000000,13.000000,44.000000,0.000000,595.075000,34405.000000,...,0.000000,3.000000,5.000000,0.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,749653.250000,34686.000000,34761.000000,34755.000000,34748.000000,15.000000,53.000000,0.000000,946.275000,34928.000000,...,1.000000,5.000000,8.000000,1.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000
max,999999.000000,35592.000000,35601.000000,35594.000000,35593.000000,20.000000,113.000000,15.000000,9999.990000,35658.000000,...,34.000000,31.000000,92.000000,112.000000,61.000000,75.000000,9.000000,5.000000,13.000000,3.000000


In [7]:
# statistical description of non-numerical columns
maindf.describe(include=['object'])

,Benefits State,Accident State,Claimant Sex,Claimant State,Claimant Marital Status,Employment Status Flag,RTW Restriction Flag,NCCI Job Code,Disability Status,SIC Group,NCCI BINatureOfLossDescription,Accident Type Group
count,160616,144713,160616,160300,160616,160616,160616,160616,139547,160616,146190,150107
unique,52,61,4,60,5,14,4,1281,4,51,15,16
top,CA,CA,M,CA,S,F,N,8810,TTD,Business Services,Strain,ATCgr6
freq,28252,24138,96719,28277,64414,134555,86423,5680,103272,14373,56071,62080


In [8]:
#review the data element types and empty values
maindf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160616 entries, 0 to 160615
Data columns (total 92 columns):
ClaimID                            160616 non-null int64
Accident DateID                    144713 non-null float64
Claim Setup DateID                 160616 non-null int64
Report To GB DateID                160616 non-null int64
Employer Notification DateID       160615 non-null float64
Benefits State                     160616 non-null object
Accident State                     144713 non-null object
Industry ID                        160616 non-null int64
Claimant Age                       160616 non-null int64
Claimant Sex                       160616 non-null object
Claimant State                     160300 non-null object
Claimant Marital Status            160616 non-null object
Number Dependents                  160616 non-null int64
Weekly Wage                        160616 non-null float64
Employment Status Flag             160616 non-null object
RTW Restriction Flag  

In [9]:
#missing data in the following

#Accident DateID	144713	non-null	float64
#Employer Notification DateID	160615	non-null	float64
#Accident State	144713	non-null	object
#Claimant State	160300	non-null	object
#Max Medical Improvement DateID	72580	non-null	float64
#Disability Status	139547	non-null	object
#NCCI BINatureOfLossDescription	146190	non-null	object
#Accident Source Code	71585	non-null	float64
#Accident Type Group	150107	non-null	object

#I would typically clean the missing data (and fill in with median as an example)but have not in this excersise
#due to time constraints

In [10]:
#lets inspect the floats
flt_df = maindf.select_dtypes(include=['float']).copy()
flt_df.head()

,Accident DateID,Employer Notification DateID,Weekly Wage,Max Medical Improvement DateID,Post Injury Weekly Wage,Accident Source Code
0,33460.0,33460.0,513.86,NaN,0.0,NaN
1,NaN,34270.0,1186.00,34726.0,0.0,6314.0
2,35043.0,35043.0,851.69,35051.0,0.0,NaN
3,33136.0,33136.0,300.00,NaN,0.0,7726.0
4,33262.0,33264.0,600.00,NaN,0.0,NaN


In [11]:
#lets inspect the objects
obj_df = maindf.select_dtypes(include=['object']).copy()
obj_df.head()




,Benefits State,Accident State,Claimant Sex,Claimant State,Claimant Marital Status,Employment Status Flag,RTW Restriction Flag,NCCI Job Code,Disability Status,SIC Group,NCCI BINatureOfLossDescription,Accident Type Group
0,OK,OK,M,OK,M,F,Y,5040,TTD,Gen. cont.,Strain,ATCgr7
1,CO,NaN,M,CO,M,F,N,5190,PPD,Gen. cont.,Inflammation,ATCgr6
2,MD,MD,M,VA,U,F,N,5183,TTD,Gen. cont.,Foreign Body,NaN
3,IL,IL,F,IL,M,P,N,8868,TTD,Misc. Manuf. Ind.,Strain,ATCgr6
4,IL,IL,F,WI,M,F,,8868,TTD,Misc. Manuf. Ind.,Contusions,ATCgr1


In [12]:
# in the interest of time, i'm going to drop the floating and object variables from the df as we would
# need to convert the float to numeric and then one hot encode the numeric values 
#to remove any numerical bias in the model 
#if we were to improve the model we would work on filling empty data
#and transforming the floating to numerical catagories then possibly one hot encoding using 

In [13]:
#while the floating, object and numerical data elements can be cleansed, 
#in the interest of time lets just focus on the boolean catagories for now as they look clean and complete
#lets create a new data frame and name it 'cleandf' with all floating and numeric variables removed

cleandf = maindf.drop(['Accident DateID',
'Claim Setup DateID',
'Report To GB DateID',
'Employer Notification DateID',
'Benefits State',
'Accident State',
'Industry ID',
'Claimant Age',
'Claimant Sex',
'Claimant State',
'Claimant Marital Status',
'Number Dependents',
'Weekly Wage',
'Employment Status Flag',
'RTW Restriction Flag',
'Max Medical Improvement DateID',
'Percent Impairment',
'Post Injury Weekly Wage',
'NCCI Job Code',
'Surgery Flag',
'Disability Status',
'SIC Group',
'NCCI BINatureOfLossDescription',
'Accident Source Code',
'Accident Type Group',
'HCPCS A Codes',
'HCPCS B Codes',
'HCPCS C Codes',
'HCPCS D Codes',
'HCPCS E Codes',
'HCPCS F Codes',
'HCPCS G Codes',
'HCPCS H Codes',
'HCPCS I Codes',
'HCPCS J Codes',
'HCPCS K Codes',
'HCPCS L Codes',
'HCPCS M Codes',
'HCPCS N Codes',
'HCPCS O Codes',
'HCPCS P Codes',
'HCPCS Q Codes',
'HCPCS R Codes',
'HCPCS S Codes',
'HCPCS T Codes',
'HCPCS U Codes',
'HCPCS V Codes',
'HCPCS W Codes',
'HCPCS X Codes',
'HCPCS Y Codes',
'HCPCS Z Codes',
'ICD Group 1',
'ICD Group 2',
'ICD Group 3',
'ICD Group 4',
'ICD Group 5',
'ICD Group 6',
'ICD Group 7',
'ICD Group 8',
'ICD Group 9',
'ICD Group 10',
'ICD Group 11',
'ICD Group 12',
'ICD Group 13',
'ICD Group 14',
'ICD Group 15',
'ICD Group 16',
'ICD Group 17',
'ICD Group 18',
'ICD Group 19',
'ICD Group 20',
'ICD Group 21',
'CPT Category - Anesthesia',
'CPT Category - Eval_Mgmt',
'CPT Category - Medicine',
'CPT Category - Path_Lab',
'CPT Category - Radiology',
'CPT Category - Surgery',
'NDC Class - Benzo',
'NDC Class - Misc (Zolpidem)',
'NDC Class - Muscle Relaxants',
'NDC Class - Stimulants', 'ClaimID'
], axis=1)


In [14]:
cleandf.head()

,Neurology Payment Flag,Neurosurgery Payment Flag,Dentist Payment Flag,Orthopedic Surgery Payment Flag,Psychiatry Payment Flag,Hand Surgery Payment Flag,Optometrist Payment Flag,Podiatry Payment Flag,Opiods Used
0,False,False,False,False,False,False,False,False,False
1,False,False,False,True,False,False,False,True,True
2,False,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False


In [15]:
#MACHINE LEARNING ALGORITHM

#We are going to use a classification model, Random Forest

X_train = cleandf.drop("Opiods Used", axis=1)
y_train = cleandf["Opiods Used"]
X_test  = cleandf.copy()

In [27]:
#Split the the original dataset to train then test using a sample of the original dataset

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [28]:
#RandomForest 
Classifier= RandomForestClassifier(n_estimators=200, random_state=10000)
Classifier.fit(X_train, y_train)
y_pred = Classifier.predict(X_test)


In [31]:
#Validate the accuracy of the model

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[22895   128]
 [ 2442   234]]
              precision    recall  f1-score   support

       False       0.90      0.99      0.95     23023
        True       0.65      0.09      0.15      2676

   micro avg       0.90      0.90      0.90     25699
   macro avg       0.78      0.54      0.55     25699
weighted avg       0.88      0.90      0.86     25699

0.8999961087980077


In [24]:
#Our accuracy is around 90%
#this means out of 100 claimants the model could accurately predict opiod use (or not) 90% of the time

In [32]:
#Present results merged with input as df
X_test["Opiods Used"] = Classifier.predict(X_test)

In [22]:
X_test

,Neurology Payment Flag,Neurosurgery Payment Flag,Dentist Payment Flag,Orthopedic Surgery Payment Flag,Psychiatry Payment Flag,Hand Surgery Payment Flag,Optometrist Payment Flag,Podiatry Payment Flag,Opiods Used
42163,False,False,False,True,False,False,False,False,False
56906,False,False,False,False,False,False,False,False,False
84551,False,False,False,False,False,False,False,False,False
152884,False,False,False,False,False,False,False,False,False
136751,False,False,False,False,False,False,False,False,False
153162,False,False,False,False,False,False,False,False,False
14846,False,False,False,False,False,False,False,False,False
25128,False,False,False,True,False,False,False,False,False
74944,False,False,False,False,False,False,False,False,False
105915,False,False,False,False,False,False,False,False,False


In [25]:
#Generate output in .csv format 
X_test.to_csv("opiodusepredictionresults.csv")